In [1]:
from pyvis.network import Network
import networkx as nx
import pandas as pd
import sqlite3
import panel as pn

# Trabalho A2
## Banco de dados - 2020.2

Apresentação relativa à segunda avaliação da disciplina Banco de Dados. Nosso trabalho foi feito em cima de dados disponibilizados no site Kaggle, que consistem em informações sobre jogadores, times e salários da NBA.Para o trabalho em cima das informações do banco, fornecido em SQLite, utilizamos a biblioteca Pandas de Python, trabalhando com dataframes para a criação dos grafos. Estes foram criados com outra biblioteca da mesma linguagem, Pyvis, que permite a criação de grafos interativos em HTML.

### Interface e repositório GitHub

https://grafozinho.vercel.app

https://github.com/AriOliv/graph_vizualization_2

### Criando a conexão com o banco

In [2]:
conn = sqlite3.connect("nba_salary.sqlite")
cur = conn.cursor()

### Vendo as tabelas

In [3]:
tb = pd.read_sql("""
SELECT a.Player, a.Tm, a.G, b.season17_18
FROM Seasons_Stats as a
INNER JOIN NBA_season1718_salary as b
ON a.PLAYER = b.Player and a.Tm = b.Tm
WHERE a.Tm != "TOT" and a.Year = 2017
""", conn)
pd.set_option('display.max_columns', None)
tb.head()

,Player,Tm,G,season17_18
0,Alex Abrines,OKC,68.0,5725000.0
1,Quincy Acy,BRK,32.0,1709538.0
2,Steven Adams,OKC,80.0,22471910.0
3,Arron Afflalo,SAC,61.0,1500000.0
4,Alexis Ajinca,NOP,39.0,4961798.0


### Entendendo os dados

In [4]:
tb = pd.read_sql("""
SELECT *
FROM Seasons_Stats
WHERE Tm != "TOT" and Year = 2017
""", conn)
tb.head(5)

,X1,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,blanl,OWS,DWS,WS,WS/48,blank2,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,24096.0,2017.0,Alex Abrines,SG,23.0,OKC,68.0,NaN,1055.0,10.1,0.560,NaN,0.144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1.2,0.9,2.1,0.095,None,NaN,NaN,NaN,NaN,134.0,341.0,0.393,NaN,NaN,NaN,40.0,94.0,0.426,0.531,44.0,49.0,0.898,NaN,NaN,86.0,40.0,NaN,NaN,NaN,114.0,406.0
1,24098.0,2017.0,Quincy Acy,PF,26.0,DAL,6.0,0.0,48.0,-1.4,0.355,NaN,0.176,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,None,-0.2,0.0,-0.1,-0.133,None,NaN,NaN,NaN,NaN,5.0,17.0,0.294,1.0,NaN,NaN,4.0,10.0,0.400,0.324,2.0,3.0,0.667,NaN,NaN,8.0,0.0,0.0,0.0,NaN,9.0,13.0
2,24099.0,2017.0,Quincy Acy,PF,26.0,BRK,32.0,1.0,510.0,13.1,0.587,NaN,0.373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0.6,0.5,1.1,0.102,None,NaN,NaN,NaN,0.0,65.0,153.0,0.425,NaN,NaN,NaN,29.0,70.0,0.414,0.542,43.0,57.0,0.754,NaN,NaN,107.0,18.0,NaN,NaN,NaN,58.0,209.0
3,24100.0,2017.0,Steven Adams,C,23.0,OKC,80.0,NaN,2389.0,16.5,0.589,NaN,0.392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,3.3,3.1,6.4,0.130,None,NaN,NaN,NaN,NaN,374.0,655.0,0.571,0.0,1.0,0.0,374.0,654.0,0.572,0.571,157.0,257.0,0.611,NaN,NaN,615.0,86.0,NaN,NaN,NaN,195.0,905.0
4,24101.0,2017.0,Arron Afflalo,SG,31.0,SAC,61.0,NaN,1580.0,9.0,0.559,NaN,0.221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1.2,0.2,1.4,0.043,None,NaN,NaN,NaN,NaN,185.0,420.0,0.440,NaN,NaN,NaN,123.0,269.0,0.457,0.514,83.0,93.0,0.892,NaN,NaN,125.0,78.0,NaN,NaN,NaN,104.0,515.0


In [5]:
df = pd.read_sql("""
SELECT a.`Player`, a.Tm, b.season17_18
FROM Seasons_Stats as a
INNER JOIN NBA_season1718_salary as b
ON a.Player = b.`Player`
WHERE a.Tm != "TOT"
""", conn)
pd.set_option('display.max_rows', None)
df.drop_duplicates(["Player", "Tm"], inplace = True)
df.sort_values("Player", inplace = True)
df.head()

,Player,Tm,season17_18
2738,A.J. Hammons,DAL,1312611.0
1822,Aaron Brooks,CHI,2116955.0
385,Aaron Brooks,HOU,2116955.0
1240,Aaron Brooks,SAC,2116955.0
806,Aaron Brooks,PHO,2116955.0


## Grafo 1

Grafo relacionando todos os jogadores e times da NBA na temporada de 2016-2017. As temporadas nos EUA, assim como o ano letivo, começa e termina em anos diferentes. Neste grafo direcionado, a direção parte dos jogadores em direção aos times. Percebemos que jogadores com diversas conexões são jogadores que trocaram de times durante a temporada, gerando conexidade no grafo. Ressaltam-se também alguns poucos grafia desconexos, dos quais não tiveram nenhuma troca no time durante a temporada (nenhuma contratação nem nenhuma venda).

In [6]:
tb = pd.read_sql("""
SELECT Player, Tm, PTS
FROM Seasons_Stats
WHERE Tm != "TOT" and Year = 2017
""", conn)

In [7]:
nba_net = Network(notebook = True, height = "750px", width = "100%", bgcolor = "#222222", font_color = "white")

#Escolhendo o tipo de física
nba_net.force_atlas_2based()


sources = tb['Player']
targets = tb['Tm']
weights = tb['PTS']

edge_data = zip(sources, targets, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    nba_net.add_node(src, src, title = src, group = dst, value = 3*w)
    nba_net.add_node(dst, dst, title = dst, shape = "image", image = "images/img2.png", value = 100*w)
    nba_net.add_edge(src, dst, value = 10*w)

neighbor_map = nba_net.get_adj_list()


#Adicionando descrição aos nós
for node in nba_net.nodes:
    node["title"] += " {}<br>".format(len(neighbor_map[node["id"]])) + " Conexões:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]]) #Diz respeito ao tamanho dos nós


nba_net.show_buttons(filter_ = None)
nba_net.show("final1.html")

## Grafo 2

Este grafo conta um pouco de história. Aqui temos três grafos desconexos que evidenciam o processo de transformação do Chicago Bulls, e a formação do famoso Dream Team. Em 1989, o Chicago Bulls, time até então de pequena relevância no cenário do basquete, começou a fazer grandes investimentos em jogadores e treinadores. Podemos perceber que, de 1989, até 1991 (ano em que o Chicago foi campeão da NBA pela primeira vez) vemos a diminuição de jogadores que passaram pelo time durante a temporada. Na NBA, em média, os times têm em torno de 12 jogadores, entre eles jogadores principais e os reservas, assim um time que passa uma temporada inteira com 12 jogadores bem evidencia o ditado: “time que está ganhando não se mexe”.

In [8]:
nba = pd.read_sql("""
SELECT Player, Tm, Year
FROM Seasons_Stats
WHERE Tm != "TOT"
""", conn)

In [9]:
nba1989 = nba[nba["Year"] == 1989 ]
nba1990 = nba[nba["Year"] == 1990 ]
nba1991 = nba[nba["Year"] == 1991 ]

nba1989_CHI = nba1989[nba1989["Tm"] == "CHI"]
nba1990_CHI = nba1990[nba1990["Tm"] == "CHI"]
nba1991_CHI = nba1991[nba1991["Tm"] == "CHI"]

nba1989_CHI["P_ano"] = nba1989["Player"] + " " + nba1989["Year"].astype(str)
nba1989_CHI["T_ano"] = nba1989["Tm"] + " " + nba1989["Year"].astype(str)
nba1990_CHI["P_ano"] = nba1990["Player"] + " " + nba1990["Year"].astype(str)
nba1990_CHI["T_ano"] = nba1990["Tm"] + " " + nba1990["Year"].astype(str)
nba1991_CHI["P_ano"] = nba1991["Player"] + " " + nba1991["Year"].astype(str)
nba1991_CHI["T_ano"] = nba1991["Tm"] + " " + nba1991["Year"].astype(str)

In [10]:
nba_net = Network(notebook=True, height="750px", width="100%", bgcolor="#222222", font_color="white")

#Escolhendo o tipo de física
nba_net.force_atlas_2based()

got_data = nba1989_CHI

sources = got_data['P_ano']
targets = got_data['T_ano']

edge_data = zip(sources, targets)

for e in edge_data:
    src = e[0]
    dst = e[1]

    nba_net.add_node(src, src, title=src, color = "#FA3113")
    nba_net.add_node(dst, dst, title=dst, shape = "image", image = "images/cb.png",)
    nba_net.add_edge(src, dst)


neighbor_map = nba_net.get_adj_list()

#Adicionando descrição aos nós

for node in nba_net.nodes:
    node["title"] += " {}<br>".format(len(neighbor_map[node["id"]])) + " Conexões:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]]) #Diz respeito ao tamanho dos nós

################################## Repetindo o processo ################################# 

got_data = nba1990_CHI

sources = got_data['P_ano']
targets = got_data['T_ano']

edge_data = zip(sources, targets)

for e in edge_data:
    src = e[0]
    dst = e[1]

    nba_net.add_node(src, src, title=src, color = "red")
    nba_net.add_node(dst, dst, title=dst, shape = "image", image = "images/cb.png",)
    nba_net.add_edge(src, dst)


neighbor_map = nba_net.get_adj_list()


for node in nba_net.nodes:
    node["title"] += " {}<br>".format(len(neighbor_map[node["id"]])) + " Conexões:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]]) 
    

################################## Repetindo o processo #################################  
    
got_data = nba1991_CHI

sources = got_data['P_ano']
targets = got_data['T_ano']

edge_data = zip(sources, targets)

for e in edge_data:
    src = e[0]
    dst = e[1]

    nba_net.add_node(src, src, title=src, color = "#FA512E")
    nba_net.add_node(dst, dst, title=dst, shape = "image", image = "images/cb.png",)
    nba_net.add_edge(src, dst)


neighbor_map = nba_net.get_adj_list()


for node in nba_net.nodes:
    node["title"] += " {}<br>".format(len(neighbor_map[node["id"]])) + " Conexões:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]])

nba_net.show("chicago.html")

## Grafo 3

Nesta última visualização, pusemos a consulta em ordem crescente em relação ao salário, e filtramos em até 10 linhas para termos os top 10 jogadores mais bem pagos da NBA nesta temporada analisada. Temos, representado por estrelas, os times conectados aos jogadores que os pertencem. O tamanho de cada aresta e dos vértices que representam jogadores corresponde ao tamanho do salário, pago a eles.

In [11]:
df_top10 = pd.read_sql("""
SELECT Player, Tm, season17_18
FROM NBA_season1718_salary
WHERE Tm != "TOT"
ORDER BY season17_18 DESC
LIMIT 10
""", conn)

nba_net = Network(notebook = True, height = "750px", width = "100%", bgcolor = "#222222", font_color = "white")

#Escolhendo o tipo de física
nba_net.force_atlas_2based()


sources = df_top10['Player']
targets = df_top10['Tm']
weights = df_top10['season17_18']

edge_data = zip(sources, targets, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    nba_net.add_node(src, src, title = src, shape = "image", image = "images/img2.png", group = dst, value = w)
    nba_net.add_node(dst, dst, title = dst, shape = "star", group = dst, value = w)
    nba_net.add_edge(src, dst, value = w)

neighbor_map = nba_net.get_adj_list()

#Adicionando descrição aos nós
for node in nba_net.nodes:
    node["title"] += " {}<br>".format(len(neighbor_map[node["id"]])) + " Conexões:<br>" + "<br>".join(neighbor_map[node["id"]])


nba_net.show("salarios.html")